In [1]:
import pandas as pd
from datetime import datetime
import requests

map_data = pd.read_csv('data_files/map_data.csv')

map_data.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_latitude,incident_type,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,40.774000,NaN,2ca11d45-8139-4c16-8af0-880d99b21e82,https://www.fire.ca.gov/incidents/2017/10/31/b...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,1.000000,Wildfire,8f61f461-552d-4538-b186-35ab030da416,https://www.fire.ca.gov/incidents/2009/5/24/pa...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,36.602575,NaN,094719ba-a47b-4abb-9ec5-a506b2b9fd23,https://www.fire.ca.gov/incidents/2013/2/24/ri...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,34.288877,NaN,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,https://www.fire.ca.gov/incidents/2013/4/20/fa...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,37.116295,NaN,357ffc13-bef9-48eb-810f-c5de851972eb,https://www.fire.ca.gov/incidents/2013/4/30/go...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False


In [2]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.3 MB/s eta 0:00:00


In [3]:
import openmeteo_requests
import requests_cache
from retry_requests import retry


cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [7]:
temperature_list = []
error_index_list = []
url = "https://archive-api.open-meteo.com/v1/archive"



for index, row in map_data.iterrows():
  try:
    lat = row['incident_latitude']
    lon = row['incident_longitude']
    start_time = row['incident_date_created']
    start_date = datetime.strptime(start_time, '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d')
    end_time = row['incident_date_last_update']
    end_date = datetime.strptime(end_time, '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d')

    params = {
    "latitude": lat,
    "longitude": lon,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": "temperature_2m"
    }

    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    temperature_list.append(hourly_temperature_2m)
  except Exception as e:
    print(f"Error at index {index}: {e}")
    error_index_list.append(index)




Error at index 121: {'error': True, 'reason': 'End-date must be larger or equals than start-date'}
Error at index 165: {'reason': 'End-date must be larger or equals than start-date', 'error': True}
Error at index 167: {'reason': 'Latitude must be in range of -90 to 90°. Given: 121.999.', 'error': True}
Error at index 171: {'error': True, 'reason': 'Latitude must be in range of -90 to 90°. Given: 120.679016.'}
Error at index 317: {'reason': 'End-date must be larger or equals than start-date', 'error': True}
Error at index 367: {'reason': 'Latitude must be in range of -90 to 90°. Given: 5487.0.', 'error': True}
Error at index 561: {'error': True, 'reason': 'Latitude must be in range of -90 to 90°. Given: 117.13874.'}
Error at index 776: {'error': True, 'reason': 'Latitude must be in range of -90 to 90°. Given: 117.33733.'}
Error at index 1175: {'error': True, 'reason': 'Latitude must be in range of -90 to 90°. Given: -120.258.'}
Error at index 1636: {'error': True, 'reason': 'Latitude mu

In [8]:
import numpy as np

mean_temperatures = []

for temp in temperature_list:
    mean_temperatures.append(np.mean(temp))


In [9]:
map_data = map_data.drop(error_index_list)

map_data.shape

(2470, 23)

In [10]:
map_data['mean_temperature'] = mean_temperatures

map_data.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_type,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired,mean_temperature
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,NaN,2ca11d45-8139-4c16-8af0-880d99b21e82,https://www.fire.ca.gov/incidents/2017/10/31/b...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False,9.712425
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,Wildfire,8f61f461-552d-4538-b186-35ab030da416,https://www.fire.ca.gov/incidents/2009/5/24/pa...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False,26.150545
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,NaN,094719ba-a47b-4abb-9ec5-a506b2b9fd23,https://www.fire.ca.gov/incidents/2013/2/24/ri...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False,18.313614
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,NaN,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,https://www.fire.ca.gov/incidents/2013/4/20/fa...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False,11.841945
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,NaN,357ffc13-bef9-48eb-810f-c5de851972eb,https://www.fire.ca.gov/incidents/2013/4/30/go...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False,21.468794


In [98]:
map_data.to_csv('data_files/temperature_map_data.csv', index = False)

In [11]:
pop_data = pd.read_csv('data_files/map_data_population_final.csv')

pop_data.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_county,Population(SUM),incident_acres_burned,incident_containment,incident_control,incident_cooperating_agencies,incident_longitude,incident_latitude,incident_type,incident_id,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active
0,Bridge Fire,Y,2018-01-09 13:46:00+00:00,2017-10-31 11:22:00+00:00,Shasta-Trinity National Forest,Shasta,181852,37.0,100.0,NaN,Shasta-Trinity National Forest,-122.309000,40.774000,NaN,2ca11d45-8139-4c16-8af0-880d99b21e82,2018-01-09 13:46:00+00:00,1/9/2018,10/31/2017,N
1,Pala Fire,Y,2020-09-16 14:07:35+00:00,2009-05-24 14:56:00+00:00,CAL FIRE San Diego Unit,San Diego,3289701,122.0,100.0,NaN,CAL FIRE San Diego Unit,1.000000,1.000000,Wildfire,8f61f461-552d-4538-b186-35ab030da416,2009-05-25 00:00:00+00:00,5/25/2009,5/24/2009,N
2,River Fire,Y,2022-10-24 11:39:23+00:00,2013-02-24 08:16:00+00:00,CAL FIRE San Bernardino Unit,Inyo,18829,407.0,100.0,NaN,"CAL FIRE San Bernardino Unit, Inyo County Sher...",-118.016510,36.602575,NaN,094719ba-a47b-4abb-9ec5-a506b2b9fd23,2013-02-28 20:00:00+00:00,2/28/2013,2/24/2013,N
3,Fawnskin Fire,Y,2013-04-22 09:00:00+00:00,2013-04-20 17:30:00+00:00,San Bernardino National Forest,San Bernardino,2180563,30.0,100.0,NaN,San Bernardino National Forest,-116.941311,34.288877,NaN,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,2013-04-22 09:00:00+00:00,4/22/2013,4/20/2013,N
4,Gold Fire,Y,2013-05-01 07:00:00+00:00,2013-04-30 12:59:00+00:00,CAL FIRE Madera-Mariposa-Merced Unit,Madera,157243,274.0,100.0,NaN,CAL FIRE Madera-Mariposa-Merced Unit,-119.635004,37.116295,NaN,357ffc13-bef9-48eb-810f-c5de851972eb,2013-05-01 07:00:00+00:00,5/1/2013,4/30/2013,N


In [12]:
pop_data = pop_data.drop(error_index_list)

pop_data.shape

(2501, 19)

In [13]:
map_data['Population'] = pop_data['Population(SUM)']

map_data.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired,mean_temperature,Population
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,2ca11d45-8139-4c16-8af0-880d99b21e82,https://www.fire.ca.gov/incidents/2017/10/31/b...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False,9.712425,181852
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,8f61f461-552d-4538-b186-35ab030da416,https://www.fire.ca.gov/incidents/2009/5/24/pa...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False,26.150545,3289701
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,094719ba-a47b-4abb-9ec5-a506b2b9fd23,https://www.fire.ca.gov/incidents/2013/2/24/ri...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False,18.313614,18829
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,https://www.fire.ca.gov/incidents/2013/4/20/fa...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False,11.841945,2180563
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,357ffc13-bef9-48eb-810f-c5de851972eb,https://www.fire.ca.gov/incidents/2013/4/30/go...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False,21.468794,157243


In [14]:
map_data['incident_dateonly_extinguished'] = pd.to_datetime(map_data['incident_dateonly_extinguished'])
map_data['incident_dateonly_created'] = pd.to_datetime(map_data['incident_dateonly_created'])



map_data['incident_days'] = map_data['incident_dateonly_extinguished'] - map_data['incident_dateonly_created']


map_data.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired,mean_temperature,Population,incident_days
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,https://www.fire.ca.gov/incidents/2017/10/31/b...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False,9.712425,181852,70 days
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,https://www.fire.ca.gov/incidents/2009/5/24/pa...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False,26.150545,3289701,1 days
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,https://www.fire.ca.gov/incidents/2013/2/24/ri...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False,18.313614,18829,4 days
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,https://www.fire.ca.gov/incidents/2013/4/20/fa...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False,11.841945,2180563,2 days
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,https://www.fire.ca.gov/incidents/2013/4/30/go...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False,21.468794,157243,1 days


In [15]:
map_data['incident_days'] = map_data['incident_days'].dt.days

map_data.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired,mean_temperature,Population,incident_days
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,https://www.fire.ca.gov/incidents/2017/10/31/b...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False,9.712425,181852,70.0
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,https://www.fire.ca.gov/incidents/2009/5/24/pa...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False,26.150545,3289701,1.0
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,https://www.fire.ca.gov/incidents/2013/2/24/ri...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False,18.313614,18829,4.0
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,https://www.fire.ca.gov/incidents/2013/4/20/fa...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False,11.841945,2180563,2.0
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,https://www.fire.ca.gov/incidents/2013/4/30/go...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False,21.468794,157243,1.0


In [16]:
map_data['month_extinguished']= map_data['incident_dateonly_extinguished'].dt.month

map_data.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired,mean_temperature,Population,incident_days,month_extinguished
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False,9.712425,181852,70.0,1.0
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False,26.150545,3289701,1.0,5.0
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False,18.313614,18829,4.0,2.0
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False,11.841945,2180563,2.0,4.0
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False,21.468794,157243,1.0,5.0


In [17]:
map_data = map_data.reset_index(drop = True)

map_data = map_data.dropna(subset=['month_extinguished'])

map_data['month_extinguished'] = map_data['month_extinguished'].astype(int)

map_data.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired,mean_temperature,Population,incident_days,month_extinguished
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False,9.712425,181852,70.0,1
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False,26.150545,3289701,1.0,5
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False,18.313614,18829,4.0,2
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False,11.841945,2180563,2.0,4
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False,21.468794,157243,1.0,5


In [18]:
map_data_dropped = map_data.drop(['incident_name', 'incident_is_final', 'incident_date_last_update',
                                  'incident_date_created', 'incident_administrative_unit_url',
                                  'incident_location', 'incident_containment', 'incident_control',
                                  'incident_cooperating_agencies', 'incident_id', 'incident_url',
                                  'incident_date_extinguished', 'incident_dateonly_extinguished',
                                  'incident_dateonly_created', 'is_active', 'notification_desired'], axis = 1)



map_data_dropped.head()

,incident_administrative_unit,incident_county,incident_acres_burned,incident_longitude,incident_latitude,incident_type,calfire_incident,mean_temperature,Population,incident_days,month_extinguished
0,Shasta-Trinity National Forest,Shasta,37.0,-122.309000,40.774000,NaN,False,9.712425,181852,70.0,1
1,CAL FIRE San Diego Unit,San Diego,122.0,1.000000,1.000000,Wildfire,True,26.150545,3289701,1.0,5
2,CAL FIRE San Bernardino Unit,Inyo,407.0,-118.016510,36.602575,NaN,True,18.313614,18829,4.0,2
3,San Bernardino National Forest,San Bernardino,30.0,-116.941311,34.288877,NaN,False,11.841945,2180563,2.0,4
4,CAL FIRE Madera-Mariposa-Merced Unit,Madera,274.0,-119.635004,37.116295,NaN,True,21.468794,157243,1.0,5


In [19]:
map_data_dropped['incident_type'] = map_data_dropped['incident_type'].fillna("Unknown")

map_data_dropped.head()

,incident_administrative_unit,incident_county,incident_acres_burned,incident_longitude,incident_latitude,incident_type,calfire_incident,mean_temperature,Population,incident_days,month_extinguished
0,Shasta-Trinity National Forest,Shasta,37.0,-122.309000,40.774000,Unknown,False,9.712425,181852,70.0,1
1,CAL FIRE San Diego Unit,San Diego,122.0,1.000000,1.000000,Wildfire,True,26.150545,3289701,1.0,5
2,CAL FIRE San Bernardino Unit,Inyo,407.0,-118.016510,36.602575,Unknown,True,18.313614,18829,4.0,2
3,San Bernardino National Forest,San Bernardino,30.0,-116.941311,34.288877,Unknown,False,11.841945,2180563,2.0,4
4,CAL FIRE Madera-Mariposa-Merced Unit,Madera,274.0,-119.635004,37.116295,Unknown,True,21.468794,157243,1.0,5


In [20]:
map_data_dropped = map_data_dropped.dropna()

map_data_dropped.shape

(1978, 11)

In [21]:
y = map_data_dropped['incident_days']

X = map_data_dropped.drop(['incident_days'], axis = 1)

In [72]:
map_data_dropped.to_csv('processed_data.csv', index=False)


In [22]:
cols = list(X.columns)

choices = {col: None for col in cols}

X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1978 entries, 0 to 2447
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   incident_administrative_unit  1978 non-null   object 
 1   incident_county               1978 non-null   object 
 2   incident_acres_burned         1978 non-null   float64
 3   incident_longitude            1978 non-null   float64
 4   incident_latitude             1978 non-null   float64
 5   incident_type                 1978 non-null   object 
 6   calfire_incident              1978 non-null   bool   
 7   mean_temperature              1978 non-null   float32
 8   Population                    1978 non-null   int64  
 9   month_extinguished            1978 non-null   int64  
dtypes: bool(1), float32(1), float64(3), int64(2), object(3)
memory usage: 148.7+ KB


In [23]:
cat_cols = X.dtypes[X.dtypes == 'object'].index
categories_list = []

for cat in cat_cols:
  categories_list.append(list(X[cat].unique()))


In [24]:
for category_column, category_values in zip(cat_cols, categories_list):
  choices[category_column] = category_values


In [25]:
encoded_data = pd.get_dummies(X)

encoded_data.head()

,incident_acres_burned,incident_longitude,incident_latitude,calfire_incident,mean_temperature,Population,month_extinguished,incident_administrative_unit_ CAL FIRE Madera-Mariposa-Merced Unit,incident_administrative_unit_ CAL FIRE Mendocino Unit,incident_administrative_unit_ CAL FIRE Riverside County Fire Department,...,incident_county_Trinity,incident_county_Tulare,incident_county_Tuolumne,incident_county_Ventura,incident_county_Yolo,incident_county_Yuba,incident_type_Fire,incident_type_Flood,incident_type_Unknown,incident_type_Wildfire
0,37.0,-122.309000,40.774000,False,9.712425,181852,1,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,122.0,1.000000,1.000000,True,26.150545,3289701,5,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,407.0,-118.016510,36.602575,True,18.313614,18829,2,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,30.0,-116.941311,34.288877,False,11.841945,2180563,4,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,274.0,-119.635004,37.116295,True,21.468794,157243,5,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(encoded_data, y, random_state = 12)


In [52]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

scaler.fit(X_train)


X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [53]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score


model1 = GradientBoostingRegressor(n_estimators = 200, learning_rate = 0.1, max_depth = 8, random_state= 22)

model1.fit(X_train_scaled, y_train)

y_pred = model1.predict(X_test_scaled)


r2 = r2_score(y_test, y_pred)

print(f"R-squared: {r2}")

R-squared: 0.8029744639644144


In [56]:
model2 = GradientBoostingRegressor(n_estimators = 220, learning_rate = 0.1, max_depth = 7, random_state= 23)

model2.fit(X_train_scaled, y_train)

y_pred = model2.predict(X_test_scaled)


r2 = r2_score(y_test, y_pred)

print(f"R-squared: {r2}")

R-squared: 0.8137347873647431


In [58]:
model3 = GradientBoostingRegressor(n_estimators = 200, learning_rate = 0.1, max_depth = 6, random_state= 24)

model3.fit(X_train_scaled, y_train)

y_pred = model3.predict(X_test_scaled)


r2 = r2_score(y_test, y_pred)

print(f"R-squared: {r2}")

R-squared: 0.8166737161918703


In [93]:
import pickle

with open('choices.pkl', 'wb') as f:
    pickle.dump(choices, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('model.pkl', 'wb') as f:
    pickle.dump(model3, f)